In [9]:
import os

from scipy import ndimage
import math
import json
import torch
import numpy as np
import pandas as pd

import cv2
import matplotlib

from matplotlib import pyplot as plt

from tqdm import tqdm
from pathlib import Path
from dotenv import load_dotenv

from utils.yolo import get_teeth_ROI
from utils.edge import tooth_isolation
from utils.preprocess import get_labels_by_image, get_image_by_labels
from utils.preprocess import xyxy_reformat, rotate_bounding_boxes, rect_include_another

matplotlib.use('module://matplotlib_inline.backend_inline')
load_dotenv()


True

In [10]:
model = torch.hub.load(r'..\YOLO', 'custom', path=r'..\YOLO\weights\8-bound.pt', source='local')
data_dir = '..' / Path(os.getenv('DATASET_DIR'))


In [11]:
target_labels = ['caries', 'filling', 'endo', 'post', 'crown', 'R.R']
# df = pd.read_csv('label_TF.csv', index_col='filename')
image_labels_df = get_image_by_labels()
# image_labels_df = df[target_labels]
# image_labels_df = get_image_by_labels(target_labels)[target_labels]

image_labels_df


,13,17,23,27,33,...,embedded,endo,filling,impacted,post
filename,,,,,,,,,,,
00006145,1,1,1,0,1,...,0,0,1,0,0
00008026,1,1,1,1,1,...,1,3,1,0,3
00008075,1,1,1,1,1,...,0,0,2,0,0
00008113,1,1,1,1,1,...,0,2,13,0,1
00008117,1,1,1,1,1,...,2,0,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...
202012050912279341994_0571026A,0,0,1,0,1,...,0,0,0,0,0
202012050912319351996_0970409A,1,1,1,1,1,...,4,0,1,0,0
202012051012069362000_0760503A,1,1,1,1,1,...,0,1,0,0,0


In [12]:
filename = image_labels_df.index[1]
filepath_image = data_dir / f'{filename}.jpg'
filepath_json = data_dir / f'{filename}.json'


In [13]:
# Image
# Inference
results = model(filepath_image)

teeth_roi = get_teeth_ROI(results)
teeth_roi_images = teeth_roi['images'][filename]
teeth_roi_split_teeth = teeth_roi['split_teeth']
teeth_roi_images


[{'flag': 'upper',
  'number': 0,
  'tooth_position': 'left',
  'org_file_name': '00008026',
  'offset': array([698, 281]),
  'image': array([[[129, 129, 129],
          [130, 130, 130],
          [133, 133, 133],
          ...,
          [135, 135, 135],
          [133, 133, 133],
          [131, 131, 131]],
  
         [[127, 127, 127],
          [129, 129, 129],
          [133, 133, 133],
          ...,
          [138, 138, 138],
          [139, 139, 139],
          [139, 139, 139]],
  
         [[127, 127, 127],
          [132, 132, 132],
          [136, 136, 136],
          ...,
          [145, 145, 145],
          [145, 145, 145],
          [146, 146, 146]],
  
         ...,
  
         [[142, 142, 142],
          [143, 143, 143],
          [137, 137, 137],
          ...,
          [117, 117, 117],
          [120, 120, 120],
          [122, 122, 122]],
  
         [[142, 142, 142],
          [139, 139, 139],
          [130, 130, 130],
          ...,
          [115, 115, 115],
   

In [14]:
teeth_roi = get_teeth_ROI(results)
teeth_roi_images = teeth_roi['images'][filename]
teeth_roi_split_teeth = teeth_roi['split_teeth']
teeth_roi_images


[{'flag': 'upper',
  'number': 0,
  'tooth_position': 'left',
  'org_file_name': '00008026',
  'offset': array([698, 281]),
  'image': array([[[129, 129, 129],
          [130, 130, 130],
          [133, 133, 133],
          ...,
          [135, 135, 135],
          [133, 133, 133],
          [131, 131, 131]],
  
         [[127, 127, 127],
          [129, 129, 129],
          [133, 133, 133],
          ...,
          [138, 138, 138],
          [139, 139, 139],
          [139, 139, 139]],
  
         [[127, 127, 127],
          [132, 132, 132],
          [136, 136, 136],
          ...,
          [145, 145, 145],
          [145, 145, 145],
          [146, 146, 146]],
  
         ...,
  
         [[142, 142, 142],
          [143, 143, 143],
          [137, 137, 137],
          ...,
          [117, 117, 117],
          [120, 120, 120],
          [122, 122, 122]],
  
         [[142, 142, 142],
          [139, 139, 139],
          [130, 130, 130],
          ...,
          [115, 115, 115],
   

# One file check

In [15]:
image_labels = {}
labels = get_labels_by_image(filepath_json, target_labels)
for target_roi in teeth_roi_images:
    flag = target_roi['flag']
    target_roi_image = target_roi['image']
    tooth_position = target_roi['tooth_position']
    im_g = cv2.cvtColor(target_roi_image, cv2.COLOR_RGBA2GRAY)
    im_g_shape = np.array(np.array(im_g.shape)[[1, 0]])

    isolation_data = tooth_isolation(im_g, flag=flag, tooth_position=tooth_position, rotation_fix=False)
    if not isolation_data:
        continue
    regions = isolation_data['crop_regions']
    theta = isolation_data['angle']
    offset = target_roi['offset']

    phi = math.radians(theta)
    for label in labels:
        xyxy = np.hstack(label['points'])  # [x, y, x, y]
        xyxy = xyxy_reformat(np.array([xyxy]))

        xyxy = xyxy - np.tile(offset, 2)
        if xyxy.min() < 0:
            continue

        xyxy = rotate_bounding_boxes(phi, im_g_shape, xyxy)
        xyxy = xyxy[0].astype(int)

        for tooth_number, region in regions.items():
            tooth_xyxy = region['xyxy']
            key = f'{filename} {tooth_number}'
            if not key in image_labels.keys():
                image_labels[key] = []
            if rect_include_another(tooth_xyxy, xyxy) > 0.5:
                image_labels[key].append(label['label'])

image_labels


{'00008026 16': [],
 '00008026 15': [],
 '00008026 14': [],
 '00008026 12': ['crown'],
 '00008026 11': ['post', 'endo', 'crown'],
 '00008026 21': ['post', 'endo', 'crown'],
 '00008026 22': ['crown'],
 '00008026 24': [],
 '00008026 25': [],
 '00008026 26': ['crown'],
 '00008026 46': [],
 '00008026 42': [],
 '00008026 41': [],
 '00008026 31': [],
 '00008026 32': ['crown'],
 '00008026 34': [],
 '00008026 35': [],
 '00008026 36': ['crown', 'post', 'endo']}

In [16]:
import warnings
warnings.filterwarnings("ignore")

image_labels = {}
for filename in tqdm(image_labels_df.index):
    filepath_image = data_dir / f'{filename}.jpg'
    filepath_json = data_dir / f'{filename}.json'

    results = model(filepath_image)

    teeth_roi = get_teeth_ROI(results)
    teeth_roi_images = teeth_roi['images'][filename]
    teeth_roi_split_teeth = teeth_roi['split_teeth']

    labels = get_labels_by_image(filepath_json, target_labels)
    for target_roi in teeth_roi_images:
        target_roi_image = target_roi['image']
        flag = target_roi['flag']
        tooth_position = target_roi['tooth_position']

        im_g = cv2.cvtColor(target_roi_image, cv2.COLOR_RGBA2GRAY)
        im_g_shape = np.array(np.array(im_g.shape)[[1, 0]])

        isolation_data = tooth_isolation(im_g, flag=flag, tooth_position=tooth_position, rotation_fix=False, save=True, filename=filename)
        if not isolation_data:
            continue
        regions = isolation_data['crop_regions']
        theta = isolation_data['angle']
        offset = target_roi['offset']

        phi = math.radians(theta)
        for label in labels:
            xyxy = np.hstack(label['points'])  # [x, y, x, y]
            xyxy = xyxy_reformat(np.array([xyxy]))

            xyxy = xyxy - np.tile(offset, 2)
            if xyxy.min() < 0:
                continue

            xyxy = rotate_bounding_boxes(phi, im_g_shape, xyxy)
            xyxy = xyxy[0].astype(int)

            for tooth_number, region in regions.items():
                tooth_xyxy = region['xyxy']
                key = f'{filename} {tooth_number}'
                if not key in image_labels.keys():
                    image_labels[key] = []
                if rect_include_another(tooth_xyxy, xyxy) > 0.5:
                    image_labels[key].append(label['label'])


  0%|          | 3/600 [00:09<30:44,  3.09s/it]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
target_roi


In [ ]:
# for tooth_number, r in region.items():
#     print(r)
# target_roi
# label
# !jupyter nbconvert --to script prepare_classify_format.ipynb

j = json.dumps(image_labels)

with open('image_labels_for_classify.json', 'w') as f:
    f.write(j)


## Check json file and jpg file pair